In [187]:
import numpy as np
import scipy.io

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LSTM, Dense, Dropout, Flatten
from keras.layers.core import Permute, Reshape
from keras import backend as K

In [188]:
left_wrist = scipy.io.loadmat('left_data.mat')
right_wrist = scipy.io.loadmat('right_data.mat')
labels  = scipy.io.loadmat('labels.mat')

In [189]:
left_wrist = np.array(left_wrist["left_data"])
right_wrist = np.array(right_wrist["right_data"])
labels = np.array(labels["labels"])

In [190]:
print(np.shape(left_wrist))
print(np.shape(right_wrist))
print(np.shape(labels))

(35774, 10)
(35774, 10)
(33874, 2)


In [191]:
print(left_wrist[0,0])
print(right_wrist[0,0])
print(labels[0,0])

1.51148520273e+12
1.51148520273e+12
1.51148526774e+12


In [192]:
start_time = labels[0,0]
offset = 1000

left_wrist[:,0] = left_wrist[:,0] - start_time - offset
right_wrist[:,0] = right_wrist[:,0] - start_time - offset

left_wrist = left_wrist[left_wrist[:,0]>=0,:];
right_wrist = right_wrist[right_wrist[:,0]>=0,:];

In [193]:
end_time = labels[-1,0]
left_wrist = left_wrist[left_wrist[:,0]<=(end_time - start_time),:];
right_wrist = right_wrist[right_wrist[:,0]<=(end_time - start_time),:];


In [194]:
print(left_wrist[0,0])
print(right_wrist[0,0])
print(labels[0,0])

29.0
29.0
1.51148526774e+12


In [195]:
labels = labels[:-1,:]

In [196]:
ppg = 0
if (ppg == 1):
    features = np.concatenate((left_wrist[:,1:10],right_wrist[:,1:10]), axis=1)
else:
    features = np.concatenate((left_wrist[:,1:7],right_wrist[:,1:7]), axis=1)

In [197]:
print(np.shape(left_wrist))
print(np.shape(right_wrist))
print(np.shape(labels))
print(np.shape(features))

(33873, 10)
(33873, 10)
(33873, 2)
(33873, 12)


In [31]:
end_time = labels[-1,0]
end_time

1511486622660.0

In [265]:
l = len(features)
time_steps = 10
n_features = 12
dataset = np.zeros((l-time_steps+1,time_steps,n_features))


In [266]:
for i in range(0,l-time_steps+1):
    dataset[i,:,:] =  features[i:i+time_steps,:]

print(np.shape(dataset))
#print(dataset[-1])

(33864, 10, 12)


In [267]:
new_labels = labels[time_steps-1:,1]
#new_labels = labels[:,1]
labels_l = new_labels.shape[0]
for i in range(0,labels_l):
    if (new_labels[i] != 0):
        new_labels[i] = 1
        
print(new_labels)
print(np.shape(new_labels))

[ 0.  0.  0. ...,  0.  0.  0.]
(33864,)


In [268]:
k = 0
while(k<labels_l):
    if (new_labels[k]==1):
        new_labels[k+1] = 1
        new_labels[k+2] = 1
        k = k+3
    else:
        k=k+1

print(new_labels)
print(np.shape(new_labels))

[ 0.  0.  0. ...,  0.  0.  0.]
(33864,)


In [269]:
set(new_labels)

{0.0, 1.0}

In [270]:
indices = np.where(new_labels == 1)[0]
np.shape(indices)

(6552,)

In [271]:
limit = 30000
X_train0 = dataset[0:limit,:]
Y_train = new_labels[0:limit]
X_valid0 = dataset[limit:,:]
Y_valid = new_labels[limit:]
num_classes = 2 # Keypress or not
# convert class vectors to binary class matrices
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_valid = keras.utils.to_categorical(Y_valid, num_classes)

In [272]:
print(np.shape(X_train0))
print(np.shape(Y_train))
print(np.shape(X_valid0))
print(np.shape(Y_valid))

(30000, 10, 12)
(30000, 2)
(3864, 10, 12)
(3864, 2)


In [273]:
#specifying hyper-parameters
batch_size = 1024
feat_map_num = 16
#win_len = 1
_, win_len, dim = X_train0.shape
#_, dim = X_train0.shape
#network_type = 'CNN'
#network_type = 'ConvLSTM'
network_type = 'LSTM'
#network_type = 'MLP'

In [274]:
def _data_reshaping(X_train, X_valid, network_type):
    _, win_len, dim = X_train.shape
    #_, dim = X_train.shape
    print(network_type)
    if network_type=='CNN' or network_type=='ConvLSTM':
        
        # make it into (frame_number, dimension, window_size, channel=1) for convNet
        X_train = np.swapaxes(X_train,1,2)
        X_valid = np.swapaxes(X_valid,1,2)

        X_train = np.reshape(X_train, (-1, dim, win_len, 1))
        X_valid = np.reshape(X_valid, (-1, dim, win_len, 1))
    if network_type=='MLP':
        X_train = np.reshape(X_train, (-1, dim*win_len))
        X_valid = np.reshape(X_valid, (-1, dim*win_len))
    
    print (np.shape(X_train))
    print (np.shape(X_train))
    return X_train, X_valid

In [275]:
def model_variant(model, network_type):
    print(network_type)
    if network_type == 'ConvLSTM':
        
        
        model.add(Permute((2, 1, 3))) # for swap-dimension
        model.add(Reshape((-1,feat_map_num*dim)))
        model.add(LSTM(32, return_sequences=False, stateful=False))
        model.add(Dropout(0.5))
    if network_type == 'CNN':
        
        model.add(Flatten())
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.5))

In [276]:
def model_conv(model):
    model.add(Conv2D(feat_map_num, kernel_size=(1, 5),
                 activation='relu',
                 input_shape=(dim, win_len, 1),
                 padding='same'))
    model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))
    model.add(Conv2D(feat_map_num, kernel_size=(1, 5), activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(1, 2)))
    model.add(Dropout(0.5))

In [277]:
def model_LSTM(model):
    model.add(LSTM(64, 
               input_shape=(win_len,dim), 
               return_sequences=True, 
               stateful=False))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=False, stateful=False))
    model.add(Dropout(0.5))

In [278]:
def model_MLP(model):
    model.add(Dense(128, activation='relu', input_shape=(dim*win_len,)))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

In [279]:
print('reshaping data for different models ...')
X_train, X_valid = _data_reshaping(X_train0, X_valid0, network_type)

reshaping data for different models ...
LSTM
(30000, 10, 12)
(30000, 10, 12)


In [280]:
print('building the model ...')
model = Sequential()
if network_type=='CNN' or network_type=='ConvLSTM':
    model_conv(model)
    model_variant(model, network_type)
if network_type=='LSTM':
    model_LSTM(model)
if network_type=='MLP': 
    model_MLP(model)
model.add(Dense(num_classes, activation='softmax'))
model.summary()

building the model ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 10, 64)            19712     
_________________________________________________________________
dropout_20 (Dropout)         (None, 10, 64)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dropout_21 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 2)                 130       
Total params: 52,866
Trainable params: 52,866
Non-trainable params: 0
_________________________________________________________________


In [281]:
print('model training ...')
epochs = 3
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          shuffle=True,       # Changed from True
          validation_data=(X_valid, Y_valid))

model training ...
Train on 30000 samples, validate on 3864 samples
Epoch 1/3
30000/30000 [==============================] - 20s - loss: 0.5098 - acc: 0.7760 - val_loss: 0.4222 - val_acc: 0.8209
Epoch 2/3
30000/30000 [==============================] - 17s - loss: 0.4509 - acc: 0.7975 - val_loss: 0.4055 - val_acc: 0.8191
Epoch 3/3
30000/30000 [==============================] - 18s - loss: 0.4383 - acc: 0.8024 - val_loss: 0.4040 - val_acc: 0.8186


In [282]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

y_pred = np.argmax(model.predict(X_valid), axis=1)
y_true = np.argmax(Y_valid, axis=1)
print('calculating confusion matrix ... ')
cf_matrix = confusion_matrix(y_true, y_pred)
print(cf_matrix)
class_wise_f1 = np.round(f1_score(y_true, y_pred, average=None)*100)*0.01
print('the mean f1 score:{:.2f}'.format(np.mean(class_wise_f1)))

calculating confusion matrix ... 
[[3116   66]
 [ 635   47]]
the mean f1 score:0.51
